In [ ]:
from map_util import *
from play import GamePlayer

In [ ]:
def fixed():
    with GamePlayer() as player:
        player.start(6)
        #sequence = 'WNWNWNWNNEEESSSSSSSSSSSSSSSSSSSSSSSSEEEEEEEEEEEEEEEEEENNEEEEEeeeeESSSSSSeEE'
        sequence = 'WNWNWNWNNEEEEEEESWSWSWSSSSSSSSSSSSSSSSSSSSSEEEEEEEEEEEEEEEEENENEEeeeeeeESSSSSSEEeEE'
        #sequence = 'WNWNWNWNNEEESSSSSSSSSSSSSSSSSSSSSSSSEEEEEEEEEEEEEEEEEENNEEESNSNSNS'

        for c in sequence:
            if c == 'N': player.move('N')
            if c == 'E': player.move('E')
            if c == 'S': player.move('S')
            if c == 'W': player.move('W')
            if c == 'n': player.use('N')
            if c == 'e': player.use('E')
            if c == 's': player.use('S')
            if c == 'w': player.use('W')
            
            if player.finished:
                print('FINISHED')
                break
            #sleep(0.1)

#fixed()

In [ ]:
with GamePlayer() as player:
    player.start(6)

    while not player.finished:
        player.explore()
        player.try_reach_exit()
        if player.finished: break
        player.try_get_health()
        player.try_get_sword()
        player.fight_enemy()
        player.try_get_key()
        player.try_open_door()
        player.move_to_pressure_plate()
        #sleep(0.1)